In [153]:
import perfplot
from typing import Union, Callable
import polars as pl
import pandas as pd
from functime.feature_extraction import tsfresh as f_ts
from tsfresh.feature_extraction import feature_calculators as tsfresh

## Global Variable

We are using the M4 dataset. We create a `pd.DataFrame`, `pl.DataFrame` and `pl.LazyFrame`. Then we define a list of dictionnary with the following structure:
<br>
(<br>
&emsp;  `<functime_function_name>`,<br>
&emsp;  `<tsfresh_function_name>`,<br>
&emsp;  `<parameters_for_functime_function>`,<br>
&emsp;   `<parameters_for_tsfresh_function>`<br>
)<br>

In [154]:
_M4_DATASET = "../data/M4_daily.parquet"

DF_PANDAS = pd.melt(pd.read_parquet(_M4_DATASET)).drop(columns=["variable"]).dropna().reset_index(drop=True)
DF_PL_EAGER = pl.from_pandas(DF_PANDAS)
DF_PL_LAZY = DF_PL_EAGER.lazy()

In [155]:
_FUNC_PARAMS_BENCH  = [
    (f_ts.absolute_energy, tsfresh.abs_energy, {}, {}),
    (f_ts.absolute_maximum, tsfresh.absolute_maximum, {}, {}),
    (f_ts.absolute_sum_of_changes, tsfresh.absolute_sum_of_changes, {}, {}),
    (f_ts.approximate_entropy, tsfresh.approximate_entropy, {"run_length": 2, "filtering_level": 0.5}, {"m": 2, "r": 0.5}),
    # (f_ts.augmented_dickey_fuller, tsfresh.augmented_dickey_fuller, "param")
    (f_ts.autocorrelation, tsfresh.autocorrelation, {"n_lags": 4}, {"lag": 4}),
    (f_ts.autoregressive_coefficients, tsfresh.ar_coefficient, {"n_lags": 4}, {"param": [{"coeff": i, "k": 4}] for i in range(5)}),
    (f_ts.benford_correlation2, tsfresh.benford_correlation, {}, {}),
    (f_ts.benford_correlation, tsfresh.benford_correlation, {}, {}),
    (f_ts.binned_entropy, tsfresh.binned_entropy, {"bin_count": 10}, {"max_bins": 10}),
    (f_ts.c3, tsfresh.c3, {"n_lags": 10}, {"lag": 10}),
    (f_ts.change_quantiles, tsfresh.change_quantiles, {"q_low": 0.1, "q_high": 0.9, "is_abs": True}, {"ql": 0.1, "qh": 0.9, "isabs": True, "f_agg": "mean"}),
    (f_ts.cid_ce, tsfresh.cid_ce, {"normalize": True}, {"normalize": True}),
    (f_ts.count_above, tsfresh.count_above, {"threshold": 0.0}, {"t": 0.0}),
    (f_ts.count_above_mean, tsfresh.count_above_mean, {}, {}),
    (f_ts.count_below, tsfresh.count_below, {"threshold": 0.0}, {"t": 0.0}),
    (f_ts.count_below_mean, tsfresh.count_below_mean, {}, {}),
    # (f_ts.cwt_coefficients, tsfresh.cwt_coefficients, {"widths": (1, 2, 3), "n_coefficients": 2},{"param": {"widths": (1, 2, 3), "coeff": 2, "w": 1}}),
    (f_ts.energy_ratios, tsfresh.energy_ratio_by_chunks, {"n_chunks": 6}, {"param": [{"num_segments": 6, "segment_focus": i} for i in range(6)]}),
    (f_ts.first_location_of_maximum, tsfresh.first_location_of_maximum, {}, {}),
    (f_ts.first_location_of_minimum, tsfresh.first_location_of_minimum, {}, {}),
    # (f_ts.fourier_entropy, tsfresh.fourier_entropy, {"n_bins": 10}, {"bins": 10}),
    # (f_ts.friedrich_coefficients, tsfresh.friedrich_coefficients, {"polynomial_order": 3, "n_quantiles": 30}, {"params": [{"m": 3, "r": 30}]}),
    (f_ts.has_duplicate, tsfresh.has_duplicate, {}, {}),
    (f_ts.has_duplicate_max, tsfresh.has_duplicate_max, {}, {}),
    (f_ts.has_duplicate_min, tsfresh.has_duplicate_min, {}, {}),
    (f_ts.index_mass_quantile, tsfresh.index_mass_quantile, {"q": 0.5}, {"param": [{"q": 0.5}]}),
    (f_ts.large_standard_deviation, tsfresh.large_standard_deviation, {"ratio": 0.25}, {"r": 0.25}),
    (f_ts.last_location_of_maximum, tsfresh.last_location_of_maximum, {}, {}),
    (f_ts.last_location_of_minimum, tsfresh.last_location_of_minimum, {}, {}),
    # (f_ts.lempel_ziv_complexity, tsfresh.lempel_ziv_complexity, {"n_bins": 5}, {"bins": 5}),
    # (f_ts.linear_trend, tsfresh.linear_trend, {}, {"params": [{"attr": "slope"}, {"attr": "intercept"}]}),
    (f_ts.longest_streak_above_mean, tsfresh.longest_strike_above_mean, {}, {}),
    (f_ts.longest_streak_below_mean, tsfresh.longest_strike_below_mean, {}, {}),
    (f_ts.mean_abs_change, tsfresh.mean_abs_change, {}, {}),
    (f_ts.mean_change, tsfresh.mean_change, {}, {}),
    (f_ts.mean_n_absolute_max, tsfresh.mean_n_absolute_max, {"n_maxima": 20}, {"number_of_maxima": 20}),
    (f_ts.mean_second_derivative_central, tsfresh.mean_second_derivative_central, {}, {}),
    (f_ts.number_crossings, tsfresh.number_crossing_m, {"crossing_value": 0.0}, {"m": 0.0}),
    (f_ts.number_cwt_peaks, tsfresh.number_cwt_peaks, {"max_width: 5"}, {"n": 5}),
    (f_ts.number_peaks, tsfresh.number_peaks, {"support": 5}, {"n": 5}),
    # (f_ts.partial_autocorrelation, tsfresh.partial_autocorrelation, "param"),
    (f_ts.percent_reoccuring_values, tsfresh.percentage_of_reoccurring_values_to_all_values, {}, {}),
    (f_ts.percent_reocurring_points, tsfresh.percentage_of_reoccurring_datapoints_to_all_datapoints, {}, {}),
    (f_ts.permutation_entropy, tsfresh.permutation_entropy, {"tau": 1,"n_dims": 3}, {"tau": 1,"dimension": 3}),
    (f_ts.range_count, tsfresh.range_count, {"lower": 0, "upper": 9, "closed": 'none'}, {"min": 0, "max": 9}),
    (f_ts.ratio_beyond_r_sigma, tsfresh.ratio_beyond_r_sigma, {"ratio": 2}, {"r": 2}),
    (f_ts.ratio_n_unique_to_length, tsfresh.ratio_value_number_to_time_series_length, {}, {}),
    (f_ts.root_mean_square, tsfresh.root_mean_square, {}, {}),
    (f_ts.sample_entropy, tsfresh.sample_entropy, {}, {}),
    (f_ts.spkt_welch_density, tsfresh.spkt_welch_density, {"n_coeffs": 10}, {"param": [{"coeff": i} for i in range(10)]}),
    (f_ts.sum_reocurring_points, tsfresh.sum_of_reoccurring_data_points, {}, {}),
    (f_ts.sum_reocurring_values, tsfresh.sum_of_reoccurring_values, {}, {}),
    (f_ts.symmetry_looking, tsfresh.symmetry_looking, {"ratio": 0.25}, {"param": [{"r": 0.25}]}),
    (f_ts.time_reversal_asymmetry_statistic, tsfresh.time_reversal_asymmetry_statistic, {"n_lags": 3}, {"lag": 3}),
    (f_ts.variation_coefficient, tsfresh.variation_coefficient, {}, {}),
    (f_ts.var_gt_std, tsfresh.variance_larger_than_standard_deviation, {}, {})
]

## Benchmark tsfresh vs functime

Benchmark function to compare the performance of functime's against tsfresh's' function. You need to provide the content the values contained in the list `_FUNC_PARAMS_BENCH`.  

In [156]:
def benchmark(f_feat: Callable, ts_feat: Callable, f_params: dict, ts_params: dict, expr: bool):
    upper_n = 24
    if f_feat.__name__ in ("binned_entropy", "approximate_entropy", "permutation_entropy", "sample_entropy"):
        upper_n = 15
    benchmark = perfplot.bench(
        setup = lambda n: (DF_PANDAS.head(n), DF_PL_EAGER.head(n)),
        kernels = [
            lambda x, _y: ts_feat(x["value"], **ts_params),
            lambda _x, y: f_feat(y["value"], **f_params) if not expr else y.select(f_feat(pl.col("value"), **f_params))
        ],
        n_range = [2**k for k in range(6, upper_n)],
        equality_check=False,
        labels=["tsfresh", "tsfresh", "functime"]
    )
    return benchmark

## Benchmark tsfresh vs functime for all the functions

Loop over `_FUNC_PARAMS_BENCH` and call `benchmark()` for each iteration.

In [157]:
def all_benchmarks(params: list[tuple], expr: bool)-> list:
    bench_df = pl.DataFrame(
        schema={
            "id": pl.Utf8,
            "n": pl.Int64,
            "tsfresh": pl.Float64,
            "functime": pl.Float64,
            "X_time": pl.Float64
        }
    )
    for x in params:
        try:
            print("Feature: {}".format(x[0].__name__))
            bench = benchmark(
                f_feat = x[0],
                ts_feat = x[1],
                f_params = x[2],
                ts_params = x[3],
                expr = expr
            )
            bench_df = pl.concat([
                pl.DataFrame({
                    "id": [x[0].__name__]*len(bench.n_range),
                    "n": bench.n_range,
                    "tsfresh": bench.timings_s[0],
                    "functime": bench.timings_s[1],
                    "X_time": bench.timings_s[0] / bench.timings_s[1]                
                }),
                bench_df]
            )
        except:
            print("Failure for feature: {}".format(x[0].__name__))
    return bench_df

In [158]:
bench_expr = all_benchmarks(params = _FUNC_PARAMS_BENCH, expr = True)

/Users/mathieucayssol/Documents/functime/.venv/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Feature: absolute_energy


Feature: absolute_maximum


Feature: absolute_sum_of_changes


Feature: approximate_entropy


...

Failure for feature: number_cwt_peaks
Feature: number_peaks


Feature: percent_reoccuring_values


Feature: percent_reocurring_points


Feature: permutation_entropy


Feature: range_count


In [ ]:
bench_series = all_benchmarks(params = _FUNC_PARAMS_BENCH, expr = False)

/Users/mathieucayssol/Documents/functime/.venv/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Feature: absolute_energy


Feature: absolute_maximum


Feature: absolute_sum_of_changes


Feature: approximate_entropy


...

Feature: sum_reocurring_values


Feature: symmetry_looking


Feature: time_reversal_asymmetry_statistic


Feature: variation_coefficient


Feature: var_gt_std


In [ ]:
def benchmark_table(df: pl.DataFrame):
    df_pivot = df.pivot(index="id", values="X_time", columns="n")
    df = df_pivot.with_columns(
        avg = df_pivot.mean(axis=1)
    )
    return df

In [ ]:
df_expr = benchmark_table(bench_expr)
df_expr.write_parquet("../benchmarks/benchmark_tsfresh_vs_functime_expr.parquet")

In [ ]:
df_series = benchmark_table(bench_series)
df_series.write_csv("../benchmarks/benchmark_tsfresh_vs_functime_series.parquet")